# Fine-tune a YOLOv8 detection model

This notebook will train a [Yolov8](https://github.com/ultralytics/ultralytics) model for tank detection using publicly available annotated images of tanks.

As the notebook will run the training with `PyTorch`, it is recommended to have GPUs available. If running in Google Colab, go to Edit > Notebook settings and select GPU hardware acceleration.

This notebook expects that the dataset created in the notebook [01_Dataset](./01_Dataset.ipynb) has been saved to disk.

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

To start, check GPU support.

In [2]:
import torch

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.3.0+cu121 (NVIDIA RTX A6000)


### Load dataset

We'll first load the dataset created on disk.

In [3]:
from pathlib import Path

dataset_dir = Path() / "dataset"

In [4]:
import fiftyone as fo

# The splits to load
splits = ["train", "val", "test"]

# Load the dataset, using tags to mark the samples in each split
dataset = fo.Dataset()
for split in splits:
    dataset.add_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
)

 100% |███████████████| 1299/1299 [1.1s elapsed, 0s remaining, 1.2K samples/s]         
 100% |█████████████████| 163/163 [131.0ms elapsed, 0s remaining, 1.3K samples/s]     
 100% |█████████████████| 162/162 [138.3ms elapsed, 0s remaining, 1.2K samples/s]     


In [5]:
session = fo.launch_app(dataset, auto=False)

Connected to FiftyOne on port 5151 at localhost.
If you are not connecting to a remote session, you may need to start a new session and specify a port
Session launched. Run `session.show()` to open the App in a cell output.


In [6]:
session.open_tab()

<IPython.core.display.Javascript object>

### Train a YoloV8 model

We can now train a YoloV8 base model on our exported dataset. We use the `yolov8n.pt` (nano) model, which is the smallest, but larger models are available from ultralytics. Simply replace `yolov8n.pt` with the [right model](https://docs.ultralytics.com/models/yolov8/#key-features).

In [7]:
from adomvi.yolo.yolo import train

train("yolov8m.pt", data=dataset_dir / "dataset.yaml", batch=64, device=[0, 1])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49.7M/49.7M [00:04<00:00, 11.7MB/s]


New https://pypi.org/project/ultralytics/8.2.30 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.28 🚀 Python-3.10.14 torch-2.3.0+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
                                                            CUDA:1 (NVIDIA RTX A6000, 48655MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=dataset/dataset.yaml, epochs=60, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, 

100%|██████████| 6.23M/6.23M [00:00<00:00, 11.5MB/s]
/home/ukemkata/miniconda3/envs/adomvi2/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


train: Scanning /home/ukemkata/workspace/adomvi2/notebooks/dataset/labels/train... 1299 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1299/1299 [00:01<00:00, 973.38it/s]


...

                   all        163        238      0.933      0.794      0.888      0.671

60 epochs completed in 0.142 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.28 🚀 Python-3.10.14 torch-2.3.0+cu121 CUDA:0 (NVIDIA RTX A6000, 48677MiB)
                                                            CUDA:1 (NVIDIA RTX A6000, 48655MiB)


/home/ukemkata/miniconda3/envs/adomvi2/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.64it/s]


                   all        163        238      0.907      0.769      0.876      0.672
Speed: 0.1ms preprocess, 2.5ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train


### Use the fine-tuned model to predict detections

Once our model is trained, we can simply use it to predict detections on our test dataset. Simply pass the path to the best model weights and the directory containing the test images to the `predict` function.

In [8]:
from pathlib import Path
from adomvi.yolo.yolo import predict

best_model = Path() / "runs/detect/train/weights/best.pt"
results = predict(best_model, source=dataset_dir / "images/test")


image 1/162 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/021bb7e1cbb12d26.jpg: 384x640 1 tank, 89.2ms
image 2/162 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/02766a4a2a22632b.jpg: 640x448 1 tank, 89.4ms
image 3/162 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/030eab769f338ba7.jpg: 352x640 1 tank, 89.2ms
image 4/162 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/046bb8fe53b32324.jpg: 448x640 2 tanks, 89.2ms
image 5/162 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/067371b2633fef26.jpg: 480x640 3 tanks, 93.5ms
image 6/162 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/06792ed46d1efc53.jpg: 480x640 1 tank, 5.0ms
image 7/162 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/06feff921dac9884.jpg: 448x640 1 tank, 5.2ms
image 8/162 /home/ukemkata/workspace/adomvi2/notebooks/dataset/images/test/08794752f8f40af8.jpg: 448x640 1 tank, 4.7ms
image 9/162 /home/ukemkata/workspace/ado

### Evaluate model predictions

We can first evaluate our model by loading the predictions in a Fiftyone dataset view of our test data. We select the images tagged as part of the test set in our dataset.

In [9]:
# The test split of the dataset
test_view = dataset.match_tags("test")

Then we can load the model predictions as fields of the images in our test view.

In [10]:
from pathlib import Path
from adomvi.yolo.utils import add_yolo_detections

prediction_field = "yolov8"
predictions_dir = Path() / "runs/detect/predict/labels"
add_yolo_detections(test_view, prediction_field=prediction_field, predictions_dir=predictions_dir, class_list=["tank"])

Now we can visualize these YOLOv8 model predictions on the samples in our dataset in the FiftyOne app:

In [11]:
session.open_tab()

<IPython.core.display.Javascript object>

Finally, we can evaluate our model's predictions and print the mAP.

In [12]:
detection_results = test_view.evaluate_detections(
    prediction_field, 
    eval_key="eval",
    compute_mAP=True,
    gt_field="ground_truth",
)

Evaluating detections...
 100% |█████████████████| 162/162 [486.5ms elapsed, 0s remaining, 333.0 samples/s]      
Performing IoU sweep...
 100% |█████████████████| 162/162 [411.6ms elapsed, 0s remaining, 393.5 samples/s]      


In [13]:
mAP = detection_results.mAP()
print(f"mAP = {mAP}")

mAP = 0.5864953046579247


In [14]:
detection_results.print_report()

              precision    recall  f1-score   support

        tank       0.83      0.84      0.84       245

   micro avg       0.83      0.84      0.84       245
   macro avg       0.83      0.84      0.84       245
weighted avg       0.83      0.84      0.84       245

